<a href="https://colab.research.google.com/github/sulthannauval/MachineLearning-Course/blob/main/Tugas-Minggu-10/10_DL_Pytorch_Sulthan_1103213065.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# Lecture 10 - DL Pytorch

---

*   Sulthan Nauval Abdillah
*   1103213065
*   Link ChatGPT : [link text](https://chatgpt.com/share/0cf5b068-e19b-4f65-a225-6081fccea107)

## Install Library

In [1]:
!pip install torch torchvision scikit-learn pandas

## Import Library

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Read Dataset

In [3]:
# Load the data (replace 'path_to_train_csv' and 'path_to_test_csv' with actual file paths)
train_df = pd.read_csv('/content/drive/MyDrive/Datasets/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Datasets/test.csv')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## Pre-Processing

In [5]:
# Selecting relevant features and target variable
features = ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
            'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
            'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
            'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea',
            'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
            'MiscVal', 'MoSold', 'YrSold']

X_train = train_df[features].values
y_train = train_df['SalePrice'].values.reshape(-1, 1)
X_test = test_df[features].values

In [6]:
# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

## Training Model

In [8]:
# Define the neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
# Instantiate the model
model = Net()

In [10]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [11]:
# Training the model
epochs = 10000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/10000], Loss: 39039340544.0000
Epoch [1001/10000], Loss: 21128448000.0000
Epoch [2001/10000], Loss: 6465397760.0000
Epoch [3001/10000], Loss: 3783099904.0000
Epoch [4001/10000], Loss: 2221480448.0000
Epoch [5001/10000], Loss: 1465652352.0000
Epoch [6001/10000], Loss: 1100104832.0000
Epoch [7001/10000], Loss: 899870592.0000
Epoch [8001/10000], Loss: 772808512.0000
Epoch [9001/10000], Loss: 675973376.0000


## Evaluation Model

In [12]:
# Predictions on test data
with torch.no_grad():
    predictions = model(X_test_tensor).numpy()

In [13]:
# Assuming test_df has an 'Id' column for submission
submission = pd.DataFrame({'Id': test_df['Id'], 'SalePrice': predictions.flatten()})
submission

,Id,SalePrice
0,1461,117557.757812
1,1462,192887.906250
2,1463,167337.500000
3,1464,180258.250000
4,1465,189969.421875
...,...,...
1454,2915,113635.726562
1455,2916,109363.273438
1456,2917,189148.968750
1457,2918,85752.679688
